In [1]:
import os
import rioxarray
import xarray as xr
import geopandas as gpd
import pandas as pd

here's a starting point comparison https://github.com/chudasama-bijal/QGIS-Plugin-Weights-of-Evidence-Model

Get training data vector data - 1s

Choose mask layer if desired

Classes
- Favourable
- Unfavourable

Get list of evidence rasters

Studentized t contrast

In [ ]:
Calculate weights
#633

Clip and make rasters the same details

Rasterise training data

Get count of data and not - e.g 1s and 0s in raster